In [1]:
pip install pandas openpyxl

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [2]:
import pandas as pd

# Load both sheets from the Excel file
file_path = "online_retail_II.xlsx"  # Adjust if filename differs
df_2009 = pd.read_excel(file_path, sheet_name="Year 2009-2010")
df_2010 = pd.read_excel(file_path, sheet_name="Year 2010-2011")

# Merge them
df = pd.concat([df_2009, df_2010], ignore_index=True)

print(f"Combined shape: {df.shape}")
df.head()


Combined shape: (1067371, 8)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [3]:
# Quick summary
print(df.info())

# Check for nulls
print(df.isnull().sum())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1067371 entries, 0 to 1067370
Data columns (total 8 columns):
 #   Column       Non-Null Count    Dtype         
---  ------       --------------    -----         
 0   Invoice      1067371 non-null  object        
 1   StockCode    1067371 non-null  object        
 2   Description  1062989 non-null  object        
 3   Quantity     1067371 non-null  int64         
 4   InvoiceDate  1067371 non-null  datetime64[ns]
 5   Price        1067371 non-null  float64       
 6   Customer ID  824364 non-null   float64       
 7   Country      1067371 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 65.1+ MB
None
Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64


In [4]:
# Add flag column for cancellations
df['is_cancelled'] = df['Invoice'].str.startswith('C', na=False)

# Convert InvoiceDate to datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors='coerce')

# Drop rows with missing Description
df = df.dropna(subset=["Description"])

# Handle Missing Customer ID
df["has_customer_id"] = df["Customer ID"].notnull()

# Remove .0 from float customer IDs and convert to string
df["Customer ID"] = df["Customer ID"].astype("Int64")  # handles NaNs too

# Total price
df["total_price"] = df["Quantity"] * df["Price"]

#Standardize column names
df.columns = [col.strip().replace(" ", "_").lower() for col in df.columns]

df.head()

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country,is_cancelled,has_customer_id,total_price
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,False,True,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,False,True,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,False,True,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,False,True,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,False,True,30.0


In [5]:
# Quick summary
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1062989 entries, 0 to 1067370
Data columns (total 11 columns):
 #   Column           Non-Null Count    Dtype         
---  ------           --------------    -----         
 0   invoice          1062989 non-null  object        
 1   stockcode        1062989 non-null  object        
 2   description      1062989 non-null  object        
 3   quantity         1062989 non-null  int64         
 4   invoicedate      1062989 non-null  datetime64[ns]
 5   price            1062989 non-null  float64       
 6   customer_id      824364 non-null   Int64         
 7   country          1062989 non-null  object        
 8   is_cancelled     1062989 non-null  bool          
 9   has_customer_id  1062989 non-null  bool          
 10  total_price      1062989 non-null  float64       
dtypes: Int64(1), bool(2), datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 84.1+ MB
None


In [7]:
import csv
cols_to_keep = [
    "invoice", "stockcode", "description", "quantity", "invoicedate",
    "price", "customer_id", "country", "is_cancelled", "total_price"
]
df_clean = df[cols_to_keep]

# Save to CSV
df_clean.to_csv("online_retail_cleaned.csv", index=False, quoting=csv.QUOTE_ALL)

print("Cleaned dataset saved successfully.")

Cleaned dataset saved successfully.


In [10]:
df_clean.head(10)

,invoice,stockcode,description,quantity,invoicedate,price,customer_id,country,is_cancelled,total_price
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United Kingdom,False,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,False,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085,United Kingdom,False,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085,United Kingdom,False,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,False,30.0
5,489434,22064,PINK DOUGHNUT TRINKET POT,24,2009-12-01 07:45:00,1.65,13085,United Kingdom,False,39.6
6,489434,21871,SAVE THE PLANET MUG,24,2009-12-01 07:45:00,1.25,13085,United Kingdom,False,30.0
7,489434,21523,FANCY FONT HOME SWEET HOME DOORMAT,10,2009-12-01 07:45:00,5.95,13085,United Kingdom,False,59.5
8,489435,22350,CAT BOWL,12,2009-12-01 07:46:00,2.55,13085,United Kingdom,False,30.6
9,489435,22349,"DOG BOWL , CHASING BALL DESIGN",12,2009-12-01 07:46:00,3.75,13085,United Kingdom,False,45.0
